In [ ]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service 
import requests
import time
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_binary  

# Base URL of your website
base_url = "https://kvk.icar.gov.in/"
base_addr = "p_prac.aspx"  # Replace with the actual URL

# Function to download a single PDF
def download_pdf(pdf_url, download_dir):
    file_name = pdf_url.split("\\")[-1]
    file_path = os.path.join(download_dir, file_name)

    with requests.get(pdf_url, stream=True, verify=False) as r:
        r.raise_for_status()  # Check for HTTP errors
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Create a download directory if it doesn't exist 
download_dir = "kvk_pop" 
os.makedirs(download_dir, exist_ok=True) 

WINDOW_SIZE = "1920,1080"
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=./new_chrome_profile")  # New profile directory
options.add_argument("--headless")
options.add_argument("--window-size=%s" % WINDOW_SIZE)
options.add_argument('--no-sandbox')
# Create a Service object
service = Service("/data2/home/sahilkamble/chromedriver") 

# Instantiate the driver
# driver = webdriver.Chrome(options=options) 
driver = webdriver.Chrome(service=service, options=options)
driver.get(base_url+base_addr)

# Find the elements
state_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlState'))
s = 22 # Skip '--Select--'
sl = len(state_select.options)
# Iterate through states
while s < sl:
    state_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlState'))
    state_option = state_select.options[s]
    st = state_option.text
    os.mkdir(os.path.join(download_dir, st))
    print(st)
    state_option.click()
    # Wait for districts to load 
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_ddlDistrict"))
    )
    district_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlDistrict'))
    # Iterate through districts
    d = 1
    dl = len(district_select.options)
    while d < dl:
        district_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlDistrict'))
        district_option = district_select.options[d]
        dt = district_option.text
        os.mkdir(os.path.join(download_dir, st, dt))
        print(dt)
        district_option.click()
        # Wait for KVKs to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ContentPlaceHolder1_ddlKvk"))
        )
        kvk_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlKvk'))
        kl = len(kvk_select.options)
        k = 1
        # Iterate through KVKs
        while k < kl :
            kvk_select = Select(driver.find_element(By.ID, 'ContentPlaceHolder1_ddlKvk'))
            kvk_option = kvk_select.options[k]
            kt = kvk_option.text
            os.mkdir(os.path.join(download_dir, st, dt, kt))
            print(kt)
            kvk_option.click()
            submit_button = driver.find_element(By.ID, 'ContentPlaceHolder1_btnSubmit')
            submit_button.click()

            # Wait for results to load & extract PDF links
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "tblSample"))
            )

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            pdf_links = [link.get('href') for link in soup.find_all('a') 
             if link.get('href') and (link.get('href').endswith('.pdf') or  link.get('href').endswith('.PDF')) and link.get('href').startswith('API')] # Modified line 
            
            print(pdf_links)

            # Download PDFs
            for link in pdf_links:
                download_pdf(base_url + link, os.path.join(download_dir, st, dt, kt))  # Adjust URL creation if needed

                time.sleep(1)  # Small delay to not overwhelm the server
            k+=1
        d+=1
    s+=1

driver.quit()


In [ ]:
import os
from pdfminer.high_level import extract_text
from langdetect import detect


def extract_text_from_pdfs(input_dir, output_dir):
    # Initialize a counter for the PDF files
    pdf_count = 0

    # Check and create the output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(".txt"):
                # Increment the counter for each PDF found
                pdf_count += 1

    # Return the total number of PDFs processed
    return pdf_count

# Set the directory containing the PDFs and the directory to save the text files
input_directory = "./kvk_text_data"  # Replace with your PDFs folder path
output_directory = "./kvk_text_data"  # Replace with your desired output folder path
number_of_pdfs = extract_text_from_pdfs(input_directory, output_directory)
print(f"Total number of PDFs: {number_of_pdfs}")

In [ ]:
Total number of PDFs: 3241
English PDFs: 1474

In [ ]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service 
import requests
import time
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_binary  

# Base URL of your website
base_url = "https://kvk.icar.gov.in/"
base_addr = "p_prac.aspx"  # Replace with the actual URL

# Function to download a single PDF
def download_pdf(pdf_url, download_dir):
    file_name = pdf_url.split("\\")[-1]
    file_path = os.path.join(download_dir, file_name)

    with requests.get(pdf_url, stream=True, verify=False) as r:
        r.raise_for_status()  # Check for HTTP errors
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Create a download directory if it doesn't exist 
download_dir = "kvk_pop" 
os.makedirs(download_dir, exist_ok=True) 

WINDOW_SIZE = "1920,1080"
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=./new_chrome_profile")  # New profile directory
options.add_argument("--headless")
options.add_argument("--window-size=%s" % WINDOW_SIZE)
options.add_argument('--no-sandbox')
# Create a Service object
service = Service("/data2/home/sahilkamble/chromedriver") 

# Instantiate the driver
# driver = webdriver.Chrome(options=options) 
driver = webdriver.Chrome(service=service, options=options)
driver.get(base_url+base_addr)


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, executable_path="/data2/home/sahilkamble/anaconda3/envs/scrapping/lib/python3.7/site-packages/chromedriver_binary")
driver.get("http://google.com/")
print ("Headless Firefox Initialized")
driver.quit()

WebDriverException: Message: 'chromedriver_binary' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [3]:
import chromedriver_binary  


In [4]:
!echo $PATH

/data2/home/sahilkamble/anaconda3/envs/scrapping/lib/python3.7/site-packages/chromedriver_binary:/data2/home/sahilkamble/anaconda3/envs/scrapping/bin:/data2/home/sahilkamble/.local/bin:/data2/home/sahilkamble/.vscode-server/cli/servers/Stable-dc96b837cf6bb4af9cd736aa3af08cf8279f7685/server/bin/remote-cli:/data2/home/sahilkamble/anaconda3/envs/scrapping/bin:/data2/home/sahilkamble/anaconda3/condabin:/data2/home/sahilkamble/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
